In [1]:
import os
import sys
os.chdir("/home/nile")
os.curdir
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME']='/home/nile/spark-2.1.0-bin-hadoop2.7'
SPARK_HOME=os.environ['SPARK_HOME']

sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.4-src.zip"))

from pyspark import SparkContext
from pyspark import SparkConf

conf=SparkConf()
conf.set("spark.executor.memory","1g")
conf.set("spark.cores.max","1")

conf.setAppName("major")

sc = SparkContext('local',conf=conf)


In [2]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
import nltk.corpus
import string
import re , math
from collections import Counter

In [3]:
#slang dict creation
f = open('slang.txt', 'r')
slang_dict = {}
for line in f:
    line = line.strip()
    parts = [p.strip() for p in line.split("=")]
    slang_dict[parts[0].lower()] = (parts[1].lower())
f.close()

In [4]:
positive = list()
f = open('/home/nile/drive/major/major2/codes/positive.txt', 'r')
for line in f:
    line = line.strip()
    positive.append(line)
f.close()

negative = list()
f = open('/home/nile/drive/major/major2/codes/negative.txt', 'r')
for line in f:
    line = line.strip()
    negative.append(line)
f.close()


In [5]:
import enchant
import editdistance 
d = enchant.Dict("en_US")
sqlContext = SQLContext(sc)
import re ,string
def preprocess(x):
    l = x.split(',')
    if(len(l)>=4):
        #convert to lower case
        text = str(l[3].decode('utf-8')).lower()
        text=text.replace('"text":', '')
        
        #2)standardising slang words
        
        l1 = text.split(" ")
        if '' in l1:
            l1.remove('')
        for i in range(len(l1)):
            if l1[i] in slang_dict:
                l1[i] = slang_dict[l1[i]]
        text=' '.join(l1)
        
        #covert @user to at_user
        text= re.sub('@[^\s]+','atuser',text)
        
        #covert #data to data
        #text = re.sub(r'#([^\s]+)', r'\1', text)
        
        #Remove additional white spaces
        text = re.sub('[\s]+', ' ', text)
        
        text  = text.replace('\\', '')
        #replace url by 'url'
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
        
         #--feature vector--#

        #1)removing punctuation marks
        text =  text.translate(string.maketrans("",""), string.punctuation)
        
         #3) removing repeated chars
        pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
        text = pattern.sub(r"\1\1", text)
        
        #3) removing words containing no
        text=re.sub(r'\w*\d\w*', '', text).strip()
       
                
        
        sid = list(str(l[1]))[5:]
        sid = int(''.join(sid))
        
        return Row(tweet=str(text),tid=sid,status=0)
        
    else:
        return Row(tweet="not there",tid=0,status=0)
        

In [6]:
rdd1 = sc.textFile('/home/nile/drive/major/major2/codes/combined_new.txt')
rdd1 = rdd1.map(preprocess)
rdd1.count()

31522

In [7]:
rddTrump = sc.textFile('/home/nile/drive/major/major2/codes/dataset-trump/final_dataset.txt')
rddTrump = rddTrump.map(preprocess)
rddTrump.count()

10855

In [8]:
## remove the undesired tweets
def rem_null(x):
    tweet = x[2]
    if x[2] != "not there" and len(tweet.split(' '))>3:
        return x

rdd1 = rdd1.filter(rem_null)
rdd1.count()

14542

In [9]:
rddTrump = rddTrump.filter(rem_null)
rddTrump.count()

5213

In [10]:
def check_word(tweet,word):
    t_list = tweet.split(' ')
    w_list = word.split(' ')
    l = len(w_list)
    for i in range(0,len(t_list)-l+1):
        if t_list[i:i+l] == w_list:
            return 1
            break
        else:
            pass
    return 0
            
    

In [11]:
########### Rules for feature extraction ################

In [12]:
def check_ordered_sar(tweet):
    t_list = tweet.split(' ')
    for term in positive: 
        w_list = term.split(' ')
        l = len(w_list)
        for i in range(0,len(t_list)-l+1):
            if t_list[i:i+l] == w_list:
                start = i+1
                for term in negative:
                    if(check_word(' '.join(t_list[i+1:]),term) == 1):
                        return 1
            else:
                pass
    return 0

In [13]:
def filter_ordered_sar(x):
    tweet = x[2]
    stat = x[0]
    sid = x[1]
    if(check_ordered_sar(tweet)==1):
        stat = 1
    return(Row(status=stat,tid=sid,tweet=tweet))
            

In [14]:
final = rdd1.map(filter_ordered_sar)
final.cache()

PythonRDD[8] at RDD at PythonRDD.scala:48

In [15]:
tweet_list = []
for x in final.collect():
    tweet = x[2]
    tweet_list.append(tweet)

In [16]:
trump = rddTrump.map(filter_ordered_sar)
for x in trump.collect():
    tweet_list.append(x[2])

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
cvector = CountVectorizer()
final_cvector = cvector.fit_transform(tweet_list)
tf_idf = TfidfTransformer(use_idf=False).fit(final_cvector)
train = tf_idf.transform(final_cvector)

In [18]:
featureVector = train.toarray()

In [19]:
labelVector = []
for x in final.collect():
    status = x[0]
    labelVector.append(status)

In [20]:
for x in trump.collect():
    labelVector.append(x[0])

In [21]:
import pickle

In [22]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [23]:
########### Naive Bayes ###########

In [24]:
labelVector = np.array(labelVector)

In [25]:
features = featureVector[0:14542:1]
label = labelVector[0:14542:1]

In [26]:
naive_bayes = GaussianNB()
naive_bayes.fit(features,label)


GaussianNB(priors=None)

In [27]:
with open('/home/nile/drive/major/major2/codes/models/naive_bayes_classifier.sav', 'wb') as fid:
    pickle.dump(naive_bayes, fid)

In [28]:
with open('/home/nile/drive/major/major2/codes/models/naive_bayes_classifier.sav', 'rb') as fid:
     nb_loaded = pickle.load(fid)

In [29]:
test_vector = featureVector[14543:19755:1]
test_label = labelVector[14543:19755:1]

In [30]:
nb_loaded.predict(test_vector)

array([0, 1, 1, ..., 0, 0, 0])

In [ ]:
########### LOGISTIC REGRESSION ################

In [34]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(random_state=1)
logistic.fit(features,label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
with open('/home/nile/drive/major/major2/codes/models/logistic.sav','wb') as fp:
    pickle.dump(logistic,fp)

In [36]:
logistic.predict(test_vector)

array([1, 1, 0, ..., 1, 1, 0])

In [47]:
############## RANDOM FOREST ################

In [37]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(random_state=1)
random_forest.fit(features,label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

In [38]:
with open('/home/nile/drive/major/major2/codes/models/random_forest.sav','wb') as fp:
    pickle.dump(random_forest,fp)

In [39]:
random_forest.predict(test_vector)

array([0, 1, 0, ..., 1, 0, 0])

In [44]:
with open('/home/nile/drive/major/major2/codes/models/logistic.sav', 'rb') as fid:
     lr_loaded = pickle.load(fid)
        
with open('/home/nile/drive/major/major2/codes/models/random_forest.sav', 'rb') as fid:
     rf_loaded = pickle.load(fid)

In [57]:
weights = list()
accuracy = list()
for cl, label1 in zip([nb_loaded, lr_loaded,rf_loaded], ['naive Bayes','logistic regression','random forest']): 
    scores = cross_validation.cross_val_score(cl, features, label, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label1))
    weights.append(scores.mean())
    accuracy.append(scores.mean())


Accuracy: 0.54 (+/- 0.04) [naive Bayes]
Accuracy: 0.86 (+/- 0.01) [logistic regression]
Accuracy: 0.85 (+/- 0.01) [random forest]


In [43]:
from sklearn.ensemble import VotingClassifier

In [47]:
for index in range(len(weights)):
    weights[index] = int(weights[index]*10)

In [48]:
weighted = VotingClassifier(estimators=[
        ('nb', nb_loaded), ('logistic regression', lr_loaded), ('random forest', rf_loaded)], voting='hard',weights=weights)
weighted.fit(features,label)

VotingClassifier(estimators=[('nb', GaussianNB(priors=None)), ('logistic regression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
        ...estimators=10, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False))],
         n_jobs=1, voting='hard', weights=[5, 8, 8])

In [49]:
with open('/home/nile/drive/major/major2/codes/models/weighted.sav','wb') as fp:
    pickle.dump(weighted,fp)

In [50]:
with open('/home/nile/drive/major/major2/codes/models/weighted.sav', 'rb') as fid:
     we_loaded = pickle.load(fid)

In [56]:
precision = list()
recall = list()

In [58]:
scores = cross_validation.cross_val_score(we_loaded, features, label, cv=5, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), "weighted ensemble"))
accuracy.append(scores.mean())

Accuracy: 0.85 (+/- 0.01) [weighted ensemble]


In [60]:
print accuracy

[0.54256900031669952, 0.85531478514064774, 0.84747590483888513, 0.85290834148716432]


In [61]:
for cl, label1 in zip([nb_loaded,lr_loaded,rf_loaded,we_loaded], ['naive Bayes','logistic regression','random forest','weighted ensemble']):
    scores = cross_validation.cross_val_score(cl, features, label, cv=5, scoring='precision')
    print("precision: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label1))
    precision.append(scores.mean())

precision: 0.21 (+/- 0.02) [naive Bayes]
precision: 0.83 (+/- 0.05) [logistic regression]
precision: 0.84 (+/- 0.04) [random forest]
precision: 0.82 (+/- 0.03) [weighted ensemble]


In [62]:
print precision

[0.21416185158943496, 0.83130615690263665, 0.83827543259522963, 0.81516453855412363]


In [63]:
for cl, label1 in zip([nb_loaded,lr_loaded,rf_loaded,we_loaded], ['naive Bayes','logistic regression','random forest','weighted ensemble']):
    scores = cross_validation.cross_val_score(cl, features, label, cv=5, scoring='recall')
    print("recall: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label1))
    recall.append(scores.mean())

recall: 0.53 (+/- 0.02) [naive Bayes]
recall: 0.29 (+/- 0.06) [logistic regression]
recall: 0.24 (+/- 0.04) [random forest]
recall: 0.28 (+/- 0.04) [weighted ensemble]


In [64]:
print precision
print recall
print accuracy

[0.21416185158943496, 0.83130615690263665, 0.83827543259522963, 0.81516453855412363]
[0.5316846468583214, 0.29070856017839564, 0.23506170855348185, 0.28083318266133628]
[0.54256900031669952, 0.85531478514064774, 0.84747590483888513, 0.85290834148716432]


In [65]:
with open('/home/nile/drive/major/major2/codes/results/precision.txt','wb') as fp:
    pickle.dump(precision,fp)

with open('/home/nile/drive/major/major2/codes/results/accuracy.txt','wb') as fp:
    pickle.dump(accuracy,fp)
    
with open('/home/nile/drive/major/major2/codes/results/recall.txt','wb') as fp:
    pickle.dump(recall,fp)